# Extract valuable data in your IDP workflow with generative AI using Amazon Bedrock
---
<div class="alert alert-block alert-info"> 
    <b>NOTE:</b>This notebook contains dependancies that are installed in a different notebook. Be sure that you have previously run the library install scripts at the top of 01-idp-genai-introduction and restart the kernel before executing this script.  
</div>

This notebook provides a hands-on instruction to using generative AI with Amazon Bedrock and the Rhubarb Python framework for intelligent document processing (IDP) tasks. IDP involves extracting valuable information and insights from documents like PDFs, forms, invoices, and scanned images. With generative AI models available in Amazon Bedrock, you can automate document processing in powerful new ways.

We'll cover several key IDP use cases:

1. Contextual content extraction: Find and extract specific information from documents based on surrounding context using Amazon Bedrock language models.

2. Structured data extraction to JSON: Use Rhubarb to extract data from documents into structured JSON formats defined by custom schemas.

3. Named entity recognition (NER): Identify and extract named entities like names, addresses, companies etc from documents with Rhubarb.

4. PII detection: Leverage out-of-the-box models in Rhubarb to detect sensitive personally identifiable information (PII) like social security numbers.

5. Schema auto-generation: Let Rhubarb automatically generate JSON schemas for data extraction based on simple natural language prompts.

By the end of this notebook, you'll understand how to apply the latest generative AI capabilities to streamline and automate key document processing workflows.


In [ ]:
#Import script libraries and create global variables
import sagemaker
import boto3
from bedrockhelper import get_response_from_claude
from textractor.parsers import response_parser
from textractor.data.constants import TextractFeatures
from textractor import Textractor

s3 = boto3.client("s3")
session = boto3.Session()
role = sagemaker.get_execution_role()
data_bucket = sagemaker.Session().default_bucket()
region = sagemaker.Session().boto_region_name
extractor = Textractor(region_name=region)
print(f"SageMaker bucket is {data_bucket}, and SageMaker Execution Role is {role}. Current region is {region}")

## 1. Contextual content extraction
---
In this section, we'll see how Amazon Bedrock's language models can extract information from documents based on the surrounding textual context.

We first load a sample employee enrollment PDF into Amazon Textract to extract the text, layout, form fields, and signatures.

In [ ]:
employee_file_name = 'samples/employee_enrollment.pdf'
s3.upload_file(Filename='../' + employee_file_name, Bucket=data_bucket, Key=employee_file_name)

document = extractor.start_document_analysis(file_source="s3://" + data_bucket + "/" + employee_file_name,
    features=[TextractFeatures.LAYOUT, TextractFeatures.FORMS, TextractFeatures.SIGNATURES],
    save_image=False)

print(document.get_text())

### Parsing the result
---
Looking at the extracted text, we can see there is valuable employee data we may want to capture, like their name listed as 
```
EMPLOYEE'S NAME
First Martha
Initial C
Last Rivera
```
Instead of writing fragile code to parse this, we can use a Bedrock language model to understand the context and extract just the information we need.

We construct a natural language prompt asking the model to find the employee's name, address, and employment status from the document text. The model then intelligently returns the relevant data in an easy-to-parse CSV format.

In [ ]:
def format_prompt(doc_text):
    return f"""

Given the document

<document>{doc_text}<document>

Find the name and address of the employee. Also find if the employee is full time or part time.
Return a CSV with headers and colunms for Name, Address, Full time(F) or Part time(P), Signature Present (Y or N)
Make sure to add quotes around colunm values in the CSV and escape any quotes inside the values
"""

prompt = format_prompt(document.get_text())
response = get_response_from_claude(prompt)
print(f"""{response[0]}\n\nThere were {response[1]} input tokens and {response[2]} output tokens used.""")

### Convert table data to SQL
---
Here we showcase how Bedrock's language understanding capabilities can parse even complex tabular data in documents. We provide the document text, a SQL schema, and an instruction to generate SQL insert statements to populate the tables with the data found in the document.

In [ ]:
account_statement_file_name = 'samples/account_statement.png'

document = extractor.analyze_document(file_source="../" + account_statement_file_name,
    features=[TextractFeatures.LAYOUT, TextractFeatures.TABLES],
    save_image=False)
print(document.get_text())

---
Bedrock handles many tedious parsing challenges for us, like dealing with text that wraps across multiple rows and cleaning up the data into the proper formats expected by the SQL database schema. This makes it vastly easier to extract structured information from documents into databases and data warehouses.

In [ ]:
doc_text = document.get_text()

prompt = f"""
You are an AI assistant tasked with generating SQL statements to insert data from a document into a set of database tables. The document contains text that may wrap across multiple rows, and you need to handle this scenario correctly when extracting data for insertion.

Follow these steps:
1. Read the entire document carefully and identify the sections containing data relevant to each table.
2. When processing text for a specific column in a table, check if the text spans multiple rows. If it does, concatenate the text from all rows to form the complete value for that column.
3. Use string manipulation functions or regular expressions to clean up the concatenated text and remove any extra whitespace, line breaks, or formatting characters.
4. If a row contains no additional data for other columns, assume that the text belongs to the previous row's corresponding column. Concatenate the text to the appropriate column value from the previous row.
5. After extracting and cleaning the data, generate the SQL INSERT statements for each table, ensuring that the values are correctly mapped to the corresponding columns.
6. Enclose string values in single quotes when inserting them into the database.
7. Handle any special characters or escape sequences as per the database's requirements.
8. Ensure that the data types and formats of the values match the corresponding column definitions in the database schema.
9. If encountering any ambiguity or missing information, make reasonable assumptions or leave the corresponding column value as NULL.
10. Test your SQL statements thoroughly with sample data to ensure accurate data insertion.


<ACCOUNT_STATEMENT>
{doc_text}
</ACCOUNT_STATEMENT>


<SQL_SCHEMA>

CREATE TABLE Account (
    AccountNumber VARCHAR(20) PRIMARY KEY,
    AccountName VARCHAR(50) NOT NULL,
    Address VARCHAR(100),
    City VARCHAR(50),
    State VARCHAR(50),
    ZipCode VARCHAR(10),
    Phone VARCHAR(20),
    Email VARCHAR(50),
    OpeningBalance DECIMAL(10, 2),
    ClosingBalance DECIMAL(10, 2),
    StatementPeriodStart DATE,
    StatementPeriodEnd DATE
);

CREATE TABLE Investments (
    InvestmentID INT AUTO_INCREMENT PRIMARY KEY,
    AccountNumber VARCHAR(20) NOT NULL,
    InvestmentName VARCHAR(50) NOT NULL,
    InvestmentCode VARCHAR(10) NOT NULL,
    Units DECIMAL(10, 4) NOT NULL,
    UnitPrice DECIMAL(10, 2) NOT NULL,
    InvestmentValue DECIMAL(10, 2) NOT NULL,
    InvestmentPercentage DECIMAL(5, 2) NOT NULL,
    FOREIGN KEY (AccountNumber) REFERENCES Account(AccountNumber)
);

CREATE TABLE InsuranceDetails (
    InsuranceID INT AUTO_INCREMENT PRIMARY KEY,
    AccountNumber VARCHAR(20) NOT NULL,
    BenefitType VARCHAR(50) NOT NULL,
    InsuranceCoverAmount DECIMAL(10, 2) NOT NULL,
    BenefitAmount DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (AccountNumber) REFERENCES Account(AccountNumber)
);

</SQL_SCHEMA>



Generate SQL statements to insert data from ACCOUNT_STATEMENT into the tables listed in SQL_SCHEMA

Think step by step in <thinking> tags.

Return SQL statements in <SQL> tags.


"""
#print(prompt)
response = get_response_from_claude(prompt)
print (f"Our prompt has {response[1]} input tokens and Claude returned {response[2]} output tokens \n\n=========================\n")
print(response[0])

### SQL Injection Risks

GenAI Security places a strong emphasis on identifying and preventing unsafe SQL statements, particularly those that involve potentially harmful actions such as 'insert', 'update' or 'delete.' 

The examples above are passing raw SQL that can be run against a database, however we should focus on creating a robust framework by Prompt Engineering that actively identifies and restricts the execution of SQL statements that could compromise data integrity or privacy and not just run unchecked SQL statements against any database

For a deeper dive into the challenges and approaches to prevent from Prompt Injection to SQL Injection attacks in text-to-SQL use cases, please read this paper: https://arxiv.org/pdf/2308.01990.pdf and take a look at the following [samples](https://github.com/aws-samples/text-to-sql-bedrock-workshop/tree/main/module_4) that will go deeper into detection and mitigation strategies

## 2. Extract data to JSON using Bedrock Multimodal

### Perform key-value extraction using custom JSON schema
---
In this section, we use the Rhubarb framework to demonstrate multimodal document understanding with Bedrock models. Rhubarb supports extracting key-value data from documents based on a custom JSON schema that defines the data fields of interest.  

We first define a JSON schema specifying fields like employee name, SSN, address, date of birth, and other information we want to extract. We then pass this schema to Rhubarb along with the document, and it intelligently extracts the corresponding data values into a JSON object matching the schema.

In [ ]:
schema = {
    "type": "object",
    "properties": {
        "employee_name": {
            "description": "Employee's Name",
            "type": "string"
        },
        "employee_ssn": {
            "description": "Employee's social security number",
            "type": "string"
        },
        "employee_address": {
            "description": "Employee's mailing address",
            "type": "string"
        },
        "employee_dob": {
            "description": "Employee's date of birth",
            "type": "string"
        },
        "employee_gender": {
            "description": "Employee's gender",
            "type": "object",
            "properties": {
                "male":{
                    "description": "Whether the employee gender is Male",
                    "type": "boolean"
                },
                "female":{
                    "description": "Whether the employee gender is Female",
                    "type": "boolean"
                }
            },
            "required": ["male", "female"]
        },
        "employee_hire_date": {
            "description": "Employee's hire date",
            "type": "string"
        },
        "employer_no": {
            "description": "Employer number",
            "type": "string"
        },
        "employment_status": {
            "type": "object",
            "description": "Employment status",
            "properties": {
                "full_time":{
                    "description": "Whether employee is full-time",
                    "type": "boolean"
                },
                "part_time": {
                    "description": "Whether employee is part-time",
                    "type": "boolean"
                }
            },
            "required": ["full_time", "part_time"]
        },
        "employee_salary_rate":{
            "description": "The dollar value of employee's salary",
            "type": "integer"
        },
        "employee_salary_frequency":{
            "type": "object",
            "description": "Salary rate of the employee",
            "properties": {
                "annual":{
                    "description": "Whether salary rate is monthly",
                    "type": "boolean"
                },
                "monthly": {
                    "description": "Whether salary rate is monthly",
                    "type": "boolean"
                },
                "semi_monthly": {
                    "description": "Whether salary rate is semi_monthly",
                    "type": "boolean"
                },
                "bi_weekly": {
                    "description": "Whether salary rate is bi_weekly",
                    "type": "boolean"
                },
                "weekly": {
                    "description": "Whether salary rate is weekly",
                    "type": "boolean"
                }
            },
            "required": ["annual", "monthly", "semi_monthly","bi_weekly","weekly"]
        }
    },
    "required": ["employee_name","employee_hire_date", "employer_no", "employment_status"]
}

In [ ]:
from rhubarb import DocAnalysis, SystemPrompts, LanguageModels


da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 boto3_session=session)
resp = da.run(message="Give me the output based on the provided schema.", output_schema=schema)
resp

### Perform table extraction using custom JSON schema
---
Continuing with Rhubarb, we demonstrate extracting complex tabular data by defining a JSON schema for the structure of a financial results table from an Amazon 10-K filing. By providing the schema to Rhubarb along with the document, it can accurately parse the table into a nested JSON object matching the specified schema.

In [ ]:
table_schema = {
  "additionalProperties": {
    "type": "object",
    "patternProperties": {
      "^(2022|2023)$": {
        "type": "object",
        "properties": {
          "Net Sales": {
            "type": "object",
            "properties": {
              "North America": {
                "type": "number"
              },
              "International": {
                "type": "number"
              },
              "AWS": {
                "type": "number"
              },
              "Consolidated": {
                "type": "number"
              }
            },
            "required": ["North America", "International", "AWS", "Consolidated"]
          },
          "Year-over-year Percentage Growth (Decline)": {
            "type": "object",
            "properties": {
              "North America": {
                "type": "number"
              },
              "International": {
                "type": "number"
              },
              "AWS": {
                "type": "number"
              },
              "Consolidated": {
                "type": "number"
              }
            },
            "required": ["North America", "International", "AWS", "Consolidated"]
          },
          "Year-over-year Percentage Growth, excluding the effect of foreign exchange rates": {
            "type": "object",
            "properties": {
              "North America": {
                "type": "number"
              },
              "International": {
                "type": "number"
              },
              "AWS": {
                "type": "number"
              },
              "Consolidated": {
                "type": "number"
              }
            },
            "required": ["North America", "International", "AWS", "Consolidated"]
          },
          "Net Sales Mix": {
            "type": "object",
            "properties": {
              "North America": {
                "type": "number"
              },
              "International": {
                "type": "number"
              },
              "AWS": {
                "type": "number"
              },
              "Consolidated": {
                "type": "number"
              }
            },
            "required": ["North America", "International", "AWS", "Consolidated"]
          }
        },
        "required": ["Net Sales", "Year-over-year Percentage Growth (Decline)", "Year-over-year Percentage Growth, excluding the effect of foreign exchange rates", "Net Sales Mix"]
      }
    }
  }
}

To save costs, we only process the page containing the table of interest. But for cases where the table location is unknown, the full document could be processed.

In [ ]:
da = DocAnalysis(file_path="../samples/amzn-10k.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 pages=[1])
resp = da.run(message="Give me data in the results of operation table from this 10-K SEC filing document. Use the schema provided.", 
              output_schema=table_schema)
resp

### Schema creation assistant
---
Rhubarb can automatically generate JSON schemas based on simple natural language prompts, saving time compared to manually defining schemas. We provide a document and ask Rhubarb to generate a schema for extracting fields like employee name, SSN, address etc.  

In [ ]:
da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 pages=[1])
resp = da.generate_schema(message="I want to extract the employee name, employee SSN, employee address, date of birth, and phone number from this document.")
resp['output']

The generated schema can then be used with the run() function to perform the actual data extraction, or it can be manually modified as needed before extracting.

In [ ]:
output_schema = resp['output']
resp = da.run(message="I want to extract the employee name, employee SSN, employee address, date of birth and phone number from this document. Use the schema provided.", 
              output_schema=output_schema)
resp

### Schema creation assistance with question rephrase 
---
Rhubarb's schema generation capabilities are further enhanced by its ability to automatically rephrase vague input questions into more specific versions tied to the actual document content.

For example, we can ask Rhubarb to "get the child's, mother's and father's details" from a birth certificate document. Rhubarb rephrases this into a more accurate question, generates a JSON schema for those data fields, and allows extracting the details directly using the rephrased question and schema.

In [ ]:

da = DocAnalysis(file_path="../samples/birth_cert.jpeg",
                modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 boto3_session=session)
resp = da.generate_schema(message="I want to get the child's, the mother's and father's details from the given document",
                          assistive_rephrase=True)
resp['output']

In [ ]:
output_schema = resp['output']['output_schema']
question = resp['output']['rephrased_question']
resp = da.run(message = question,
              output_schema = output_schema)
resp

### Perform Named Entity Recognition
---
This section showcases using Rhubarb to automatically identify and extract named entities like names, locations, organizations etc. from documents. Rhubarb provides access to pre-trained models capable of detecting 50 common entity types out-of-the-box.

We simply specify the entities we want to detect, like PERSON and ADDRESS, and pass the document to Rhubarb's run_entity() method. It returns all detected entity values and their categories.

In [ ]:
from rhubarb import Entities

da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                 boto3_session=session,
                modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 pages=[1,3])
resp = da.run_entity(message="Extract all the specified entities from this document.", 
                     entities=[Entities.PERSON, Entities.ADDRESS])
resp

### Perform PII Recognition
---
Similar to named entity recognition, Rhubarb also supports automatically detecting sensitive personally identifiable information (PII) like social security numbers, credit cards, and addresses directly from documents.

We use the run_entity() method again, but this time specifying PII entities like SSN and ADDRESS that we want to detect and extract. Rhubarb handles automatically identifying and isolating this sensitive PII data.

If the input data is just text and doesn't have to be extracted using a FM, we can leverage [Bedrock Guardrails](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-sensitive-filters.html) to do PII detection 

In [ ]:
da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 pages=[1,3])
resp = da.run_entity(message="Extract all the specified entities from this document.", 
                     entities=[Entities.SSN, Entities.ADDRESS])
resp

## Cleanup
---
Finally, the code demonstrates how to clean up by deleting the sample files uploaded to Amazon S3 earlier in the notebook.

In [ ]:
s3.delete_object(Bucket=data_bucket, Key=employee_file_name)
s3.delete_object(Bucket=data_bucket, Key=account_statement_file_name)